In [7]:
import os
from facenet_pytorch import MTCNN, InceptionResnetV1, fixed_image_standardization
import torch
from torch.utils.data import DataLoader
from torchvision.utils import make_grid
from torchvision import transforms as v2
from torchvision.datasets import ImageFolder
import tqdm
import numpy as np
import warnings
import matplotlib.pyplot as plt

from src.utils import *
from src.modules import *

warnings.filterwarnings("ignore")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [11]:
transforms = v2.Compose(
    [v2.RandomHorizontalFlip(),
        v2.RandomRotation(15),
        v2.RandomResizedCrop(160, scale=(0.8, 1.0)),
        v2.ColorJitter(brightness=0.1, contrast=0.1, saturation=0.1, hue=0.1),
        v2.GaussianBlur(kernel_size=3, sigma=(0.1, 2.0)),
        v2.RandomGrayscale(p=0.1),
        v2.RandomAffine(degrees=0, translate=(0.1, 0.1), scale=(0.9, 1.1)),
        v2.Resize((160, 160)),
     v2.ToTensor(),
     fixed_image_standardization]
)

dataset = ImageFolder("FaceDataset/Train_cropped", transform=transforms)
loader = DataLoader(dataset, batch_size=6, shuffle=True)

In [21]:
class FaceNetClassifier(nn.Module):
    def __init__(self, num_classes):
        super(FaceNetClassifier, self).__init__()
        self.extractor = InceptionResnetV1(pretrained="casia-webface", classify=False)
        self.fc = nn.Sequential(
            nn.Linear(512, 4096),
            nn.Dropout(0.8),
            nn.Linear(4096, 256),
            nn.Dropout(0.2),
            nn.Linear(256, num_classes)
        )
    
    def forward(self, x):
        x = self.extractor(x)
        x = self.fc(x)
        return x

In [22]:
facenet = FaceNetClassifier(num_classes=len(dataset.classes)).to(device)

epochs = 100
facenet_params = list(facenet.extractor.parameters())
classifier_params = list(facenet.fc.parameters())
optimizer = torch.optim.Adam([
    {'params': facenet_params, 'lr': 1e-5},
    {'params': classifier_params, 'lr': 1e-3}
])
criterion = torch.nn.CrossEntropyLoss()

early_stopping = EarlyStopping(patience=5, verbose=True)

train_losses = []
train_accuracies = []
val_losses = []
val_accuracies = []

best_train_loss = float('inf')
best_train_acc = 0.0

for epoch in range(epochs):
    facenet.train()
    train_loss = 0.0
    correct = 0
    total = 0
    for x, y in (loader):
        x, y = x.to(device), y.to(device)
        optimizer.zero_grad()

        y_pred = facenet(x)

        loss = criterion(y_pred, y)
        loss.backward()

        optimizer.step()
        train_loss += loss.item() * y.size(0) 
        _, predicted = torch.max(y_pred, 1) 
        correct += (predicted == y).sum().item()
        total += y.size(0)
    train_loss /= total 
    train_acc = correct / total 

    train_losses.append(train_loss)
    train_accuracies.append(train_acc)
    early_stopping(train_loss, train_acc)
    if train_loss < best_train_loss:
        best_val_loss = train_loss
        best_val_acc = train_acc
        torch.save({
            'facenet_state_dict': facenet.state_dict(),
        }, 'models/best_model.pth')
    if early_stopping.early_stop:
        print("Early stopping")
        break
    torch.save({
        'facenet_state_dict': facenet.state_dict(),
    }, 'models/last_model.pth')
    torch.save({
        "losses": train_losses,
        "accuracies": train_accuracies,
    }, 'models/train_metrics.pth')
    print(f"Epoch: {epoch} | Train Loss: {train_loss:.4f}, Train Accuracy: {train_acc:.4f}")



  0%|          | 0.00/111M [00:00<?, ?B/s]

Loss improved to 3.077087.
Accuracy did not improve. Counter: 1/5
Epoch: 0 | Train Loss: 3.0771, Train Accuracy: 0.0000
Loss improved to 2.996736.
Accuracy improved to 0.238095.
Epoch: 1 | Train Loss: 2.9967, Train Accuracy: 0.2381
Loss improved to 2.871979.
Accuracy improved to 0.428571.
Epoch: 2 | Train Loss: 2.8720, Train Accuracy: 0.4286
Loss improved to 2.780027.
Accuracy improved to 0.571429.
Epoch: 3 | Train Loss: 2.7800, Train Accuracy: 0.5714
Loss improved to 2.649900.
Accuracy improved to 0.761905.
Epoch: 4 | Train Loss: 2.6499, Train Accuracy: 0.7619
Loss improved to 2.460092.
Accuracy did not improve. Counter: 1/5
Epoch: 5 | Train Loss: 2.4601, Train Accuracy: 0.7619
Loss improved to 2.241595.
Accuracy improved to 0.904762.
Epoch: 6 | Train Loss: 2.2416, Train Accuracy: 0.9048
Loss improved to 1.981064.
Accuracy did not improve. Counter: 1/5
Epoch: 7 | Train Loss: 1.9811, Train Accuracy: 0.9048
Loss improved to 1.685542.
Accuracy did not improve. Counter: 1/5
Epoch: 8 | Tra

In [23]:
test_transforms = v2.Compose(
    [v2.Resize((160, 160)),
     v2.ToTensor(),
     fixed_image_standardization]
)

test_dataset = ImageFolder("FaceDataset/Test_cropped", transform=transforms)
test_loader = DataLoader(test_dataset, batch_size=8, shuffle=False)

In [24]:
test_acc = 0.0
test_loss = 0.0

checkpoint = torch.load(r'models\last_model.pth')
facenet.load_state_dict(checkpoint['facenet_state_dict'])

facenet.eval()

correct = 0
total = 0
for x, y in test_loader:
    x, y = x.to(device), y.to(device)
    y_pred = facenet(x)
    _, predicted = torch.max(y_pred, 1)
    correct += (predicted == y).sum().item()
    total += y.size(0)
test_acc = correct / total
print(f"Test Acc: {test_acc*100}")


Test Acc: 25.53911205073996
